## Web Scraping Forbes Global 500 (2017) Data 

In [1]:
import awscli
import sys
import selenium
import unittest
import boto3
import pandas as pd
import tabulate
import time
import requests
import csv
import dataframe
from bs4 import BeautifulSoup
from tabulate import tabulate
from selenium import webdriver

In [2]:
browser = webdriver.Chrome(r"C:\Users\Jeeho\AppData\Local\Programs\Python\Python36\chromedriver.exe")
browser.get('https://fortune.com/global500/2017/search/')

In [3]:
#Find EIN filed
inputElement = browser.find_element_by_xpath("//*[@id='content']/div/div[2]/div/div[2]/div/div[2]/span[2]/select")
inputElement.send_keys('100 Rows')

In [4]:
#identify the table to scrape
table = browser.find_element_by_css_selector('div.rt-tbody')

In [5]:
datetime = time.strftime("%Y%m%d-%H%M%S")
filedirectory = 'C:/Users/Jeeho/Documents/Python Scripts/Capstone project/Web_Scrap/'
filename = 'fg500_2017_'
datetime = time.strftime("%Y%m%d%H%M%S")
file = "%s%s%s.csv"%(filedirectory,filename,datetime)

In [7]:
#create empty dataframe Test 2 : Included For Loop for next pages 
data =[]

#Use For Loop for Index

index = 1
while True:
    if index > 5:
        break
    try:
        print("Scraping Page number " + str(index))
        index = index + 1
        for row in table.find_elements_by_css_selector('div.rt-tr'):
            cols = data.append([cell.text for cell in row.find_elements_by_css_selector('div.rt-td.searchResults__cell--2Y7Ce')])

        time.sleep(4)
        next_button = browser.find_element_by_css_selector('div.-next')
        next_button.click()
    except Exception as e:
        print(e)
        break


data1 = pd.DataFrame(data, columns = ['rank','company','revenues','revenue % change', 'profits($M)','assets($M)','profits % change','employees','change in rank 500'])

data1['year']=2017

print(data1)

Scraping Page number 1
Scraping Page number 2
Scraping Page number 3
Scraping Page number 4
Scraping Page number 5
    rank                                       company  revenues  \
0      1                                       Walmart  $485,873   
1      2                                    State Grid  $315,199   
2      3                                 Sinopec Group  $267,518   
3      4                      China National Petroleum  $262,573   
4      5                                  Toyota Motor  $254,694   
5      6                                    Volkswagen  $240,264   
6      7                             Royal Dutch Shell  $240,033   
7      8                            Berkshire Hathaway  $223,604   
8      9                                         Apple  $215,639   
9     10                                   Exxon Mobil  $205,004   
10    11                                      McKesson  $198,533   
11    12                                            BP  $186,606   
1

In [9]:
data1.to_csv(file, header=True, line_terminator='\n')

In [ ]:
from botocore.client import Config
from io import StringIO

ACCESS_KEY_ID = 'your_access_key_id'
ACCESS_SECRET_KEY = 'your_access_secret_key'
BUCKET_NAME = 'ceo-culture-sagemaker'

csv_buffer =StringIO()
data1.to_csv(csv_buffer)

s3 = boto3.resource(
        's3',
        aws_access_key_id = ACCESS_KEY_ID,
        aws_secret_access_key=ACCESS_SECRET_KEY,
        config=Config(signature_version='s3v4')
        )
s3.Bucket(BUCKET_NAME).put_object(Key='fg500_2017_20200414020411.csv', Body = csv_buffer.getvalue())

print("Uploaded")